In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas
#noms=('seismic', 'seismoacoustic', 'shift', 'genergy', 'gpuls', 'gdenergy', 'gdpuls', 'ghasard', 'nbumps2', 'nbumps3', 'nbumps4,'nbumps5',  )

dataframe= pandas.read_csv('/content/drive/MyDrive/séismique de charbon/seismic-bumps.csv')

In [3]:
dataframe

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,ghazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
0,a,a,N,15180,48,-72,-72,a,0,0,0,0,0,0,0,0,0,0,0
1,a,a,N,14720,33,-70,-79,a,1,0,1,0,0,0,0,0,2000,2000,0
2,a,a,N,8050,30,-81,-78,a,0,0,0,0,0,0,0,0,0,0,0
3,a,a,N,28820,171,-23,40,a,1,0,1,0,0,0,0,0,3000,3000,0
4,a,a,N,12640,57,-63,-52,a,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573,b,a,W,81410,785,432,151,b,0,0,0,0,0,0,0,0,0,0,0
2574,b,a,W,42110,555,213,118,a,0,0,0,0,0,0,0,0,0,0,0
2575,b,a,W,26960,540,101,112,a,0,0,0,0,0,0,0,0,0,0,0
2576,a,a,W,16130,322,2,2,a,0,0,0,0,0,0,0,0,0,0,0


In [4]:
donnees= dataframe.values
d= donnees[:, 0:18]
t= donnees[:, 18]

In [7]:
import numpy as np

def binarisation (d, indice, values):
  B= np.zeros((len(d), len(values)))
  for i in range(len(d)):
    for j in range(len(values)):
      if d[i,indice]== values[j]:
        B[i,j]=1

  return B
A0= binarisation(d, 0, ['a', 'b', 'c','d'])
A1= binarisation(d, 1, ['a', 'b', 'c','d'])
A2= binarisation(d, 2, ['W', 'N'])
A7= binarisation(d, 7, ['a', 'b', 'c','d'])

x=np.concatenate((A0, A1, A2, d[:, 3:7],A7, d[:, 8:18]), axis=1)
y= t.astype('int')




In [14]:
len(x[y==0]), len(x[y==1])

(2408, 170)

In [15]:
len(x[y==0])/ ((len(x[y==1])) + len(x[y==0]))

0.9340574088440652

In [17]:
import random
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.3, random_state= random.seed())

In [19]:
from sklearn.ensemble import RandomForestClassifier
my_model= RandomForestClassifier(n_estimators=10)
my_model.fit(x_train, y_train)
predictions=my_model.predict(x_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9198966408268734

In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix (y_test, predictions)

array([[710,   8],
       [ 54,   2]])

In [25]:
from sklearn.metrics import f1_score
f1_score(y_test, predictions)

0.06060606060606061

In [28]:
nb= 170 #pour en avoir autant de samples que dans la classe minoritaire
from sklearn.utils import resample
echantillon_majoritaire= resample(x[y==0], replace=False, n_samples= nb, random_state= random.seed())
#replace means "echantillonage sans remise lorsque = False"
#on combine la classe minoritaire avec le sous-echantillon de la classe majoritaire
x1= np.concatenate((echantillon_majoritaire, x[y==1]), axis=0)
y1= np.concatenate((np.ones(nb), np.zeros(170)), axis=0)

In [30]:
x1_train, x1_test, y1_train, y1_test= train_test_split(x1, y1, test_size=0.3, random_state= random.seed())
my_model.fit(x1_train, y1_train)
newpredictions= my_model.predict(x1_test)

In [31]:
f1_score(y1_test, newpredictions)

0.5490196078431373

In [32]:
confusion_matrix(y1_test, newpredictions)

array([[28, 22],
       [24, 28]])

In [33]:
poids= 2578/nb
p= (1-y1_test)*poids
p= p+ y1_test
f1_score(y1_test, newpredictions, sample_weight=p)

0.13538881620114906

In [34]:
confusion_matrix(y1_test, newpredictions, sample_weight=p)

array([[424.61176471, 333.62352941],
       [ 24.        ,  28.        ]])